In [1]:
import pandas as pd
import numpy as np
from numpy.random import choice
import my_pickle as mp
import matplotlib.pyplot as plt

from importlib import reload

In [ ]:
reload(mp)

# LOAD DATA FRAMES

In [2]:
# Load User Data
user_df = mp.unjson_it('data_user')
user_df = mp.reinstate_date(user_df,['activeAt', 'available', 'birthday','created','updated'])
user_df.head(2)

,I_count,I_ratio,about,activeAt,age,amenities,available,birthday,college,created,...,picture,question_count,question_ratio,sentence_count,sentence_ratio,smokingOk,term,type,updated,work
00DMWuKDFs,1,0.038462,I'm really busy with work,2017-10-17 22:30:39.464999936,21,None,NaT,1997-01-01,None,2017-10-15 01:04:32.383,...,True,0,0.0,0,0.0,1,NaN,None,2018-01-12 22:15:57.071,VIP Valet
00kOFrnYJu,0,NaN,,2017-11-24 22:03:24.855000064,21,"[Laundry, A/C, Parking, Dishwasher, Private Ba...",2034-01-10 17:02:07,1997-01-01,DeVry University,2017-11-24 19:53:10.262,...,True,0,NaN,0,NaN,1,8.0,None,2018-01-12 03:47:41.473,None


In [36]:
# Load Conversation Data
convo_df = mp.unjson_it('data_convo')
print("Make sure we're not using ECT data")
print(convo_df.timestamp.max())
convo_df = convo_df.set_index('conversation_id')
convo_df.head(2)

Make sure we're not using ECT data
2017-10-02 01:35:58.644000


,convo_length,len_receiver,len_sender,mid_receiver,mid_sender,response,timestamp,uid_receiver,uid_sender
conversation_id,,,,,,,,,
1Jqbth3nqH0EWNOmyQmW,1,NaN,17.0,None,-Kt7F-lKnoanl54lMnUO,False,2017-09-03 14:58:30.759,0EWNOmyQmW,1Jqbth3nqH
caEwxVJXPTGyR6cCT5K8,1,NaN,9.0,None,-KhP4gs1cj3-MfN7aZkb,False,2017-04-10 22:58:15.738,caEwxVJXPT,GyR6cCT5K8


In [4]:
# Import Message DF
message_df = mp.unjson_it('data_message')
print("Make sure we're not using ECT data")
print(convo_df.timestamp.max())
message_df.head(2)

Make sure we're not using ECT data
2017-10-02 01:35:58.644000


,const,conversation_id,date,emailAttempted,emailed,flag,imageURL,message_id,read,readBy,s_uid,text_length,timestamp,uid
0,1,-KdmSL0izd_44wrniy-W,2017-02-24 23:30:04.690,,,False,,-KdmSRA-1a7niRVp5s83,True,"{'1': '6y3bPICtHM', '0': 'OBFaJBVxWC'}",NaN,3,2017-02-24 23:30:04.690,8Ws3QWmC4B
1,1,-KdmSL0izd_44wrniy-W,2017-02-27 16:39:19.164,,,False,,-Ke-RBSDV6VsM2fE2cwu,True,"{'1': '6y3bPICtHM', '0': '8Ws3QWmC4B'}",NaN,3,2017-02-27 16:39:19.164,OBFaJBVxWC


# GET USER DATA FOR SENDER AND RECEIVER

In [37]:
user_df['flag'] = True
old_names = user_df.columns
new_names = {old:old+'_sender' for old in old_names}
master_df = convo_df.join(user_df, on='uid_sender').join(user_df, on='uid_receiver', rsuffix="_receiver").rename(index=str, columns = new_names)
master_df.head()

,convo_length,len_receiver,len_sender,mid_receiver,mid_sender,response,timestamp,uid_receiver,uid_sender,I_count_sender,...,question_count_receiver,question_ratio_receiver,sentence_count_receiver,sentence_ratio_receiver,smokingOk_receiver,term_receiver,type_receiver,updated_receiver,work_receiver,flag_receiver
conversation_id,,,,,,,,,,,,,,,,,,,,,
1Jqbth3nqH0EWNOmyQmW,1,NaN,17.0,None,-Kt7F-lKnoanl54lMnUO,False,2017-09-03 14:58:30.759,0EWNOmyQmW,1Jqbth3nqH,3.0,...,0.0,0.0,6.0,0.015789,0.0,NaN,None,2018-01-12 14:10:21.838,Insightly,True
caEwxVJXPTGyR6cCT5K8,1,NaN,9.0,None,-KhP4gs1cj3-MfN7aZkb,False,2017-04-10 22:58:15.738,caEwxVJXPT,GyR6cCT5K8,0.0,...,0.0,0.0,10.0,0.022272,1.0,NaN,None,2018-01-12 10:40:00.228,Shamblott Family Dentistry,True
caZHBg3cEbGhtafu1Ef5,1,NaN,36.0,None,-KnNpug-1BJr75KDw7lH,False,2017-06-24 06:43:51.751,caZHBg3cEb,Ghtafu1Ef5,0.0,...,0.0,0.0,1.0,0.005319,1.0,12.0,private,2018-01-12 12:48:19.532,None,True
ccY3lAJp8URACdaEvb0s,1,NaN,24.0,None,-KuB6nmPi4W96oF6J_pc,False,2017-09-16 19:16:51.128,ccY3lAJp8U,RACdaEvb0s,8.0,...,0.0,0.0,0.0,0.000000,1.0,24.0,private,2018-01-12 16:26:34.803,The Black Cat,True
ccYEEH0Ug7FH2hR293u1,2,26.0,16.0,-KindiRhJWOmeQ0Nt43Q,-KimFk44pXMwVVvWqAVl,True,2017-04-28 01:53:07.603,FH2hR293u1,ccYEEH0Ug7,0.0,...,0.0,0.0,5.0,0.020000,0.0,12.0,private,2018-01-12 11:18:39.313,None,True


# DROP USERS THAT AREN'T ON THE USER_DF

In [38]:
# Investigate
master_df.flag_receiver = master_df.flag_receiver.apply(lambda x: x if x==True else False)
print(len(master_df))
print(master_df.flag_receiver.sum())
len(master_df[master_df.flag_receiver == False])

11404
11127


277

In [39]:
master_df = master_df[master_df.flag_receiver]
len(master_df)

11127

# ADD FEATURES

In [40]:
def roommate_rules(roommates):
    if roommates > 4: return 3
    elif roommates > 1: return 2
    elif roommates > 0: return 1
    else: return 0

def get_rent_range(row):
    max1 = row.maxCost_sender
    max2 = row.maxCost_receiver
    min1 = row.minCost_sender
    min2 = row.minCost_receiver
    if max1*max2*min1*min2 > 0:
        upper = min(max1,max2)
        lower = max(min1,min2)
        if upper-lower > 0: return upper-lower
        else: return 0
    else: return 175
    
def get_urgency_receiver(row):
    try:
        return row.available_receiver-row.timestamp
    except:
        return None
    
def get_urgency_sender(row):
    try:
        return row.available_sender-row.timestamp
    except:
        return None
    
def my_distance(row):
    a = row.location_receiver
    b = row.location_sender
    try: 
        one = b[0]-a[0]
        two = b[1]-a[1]
        return (one**2+two**2)**(.5)
    except:
        return None

def feature_time(df):
    df['age_dif'] = abs(df.age_sender-df.age_receiver)
    df['same_gender'] = df.gender_sender==df.gender_receiver
    df['same_relate'] = df.inRelationship_sender == df.inRelationship_receiver
    df['same_clean'] = df.isClean_sender == df.isClean_receiver
    df['same_night'] = df.isNight_sender == df.isNight_receiver
    df['same_student'] = df.isStudent_sender == df.isStudent_receiver
    df['same_student'] = df.isStudent_sender == df.isStudent_receiver
    df['same_smoking'] = df.smokingOk_sender == df.smokingOk_receiver
    df['same_type'] = df.type_sender == df.type_receiver
    df['same_term'] = df.term_sender == df.term_receiver
    df['same_work'] = df.work_sender == df.work_receiver
    df['same_city'] = df.hometownCity_sender == df.hometownCity_receiver
    df['same_state'] = df.hometownState_sender == df.hometownState_receiver
    df['same_country'] = df.hometownCountry_sender == df.hometownCountry_receiver
    df['same_college'] = df.college_sender == df.college_receiver
    df['roommate_similarity'] = abs(df.numRoommates_sender.apply(lambda x: roommate_rules(x))
                                  -df.numRoommates_receiver.apply(lambda x: roommate_rules(x)))

    df['hobbies_receiver'] = df.hobbies_receiver.apply(lambda x: set(x) if isinstance(x,list) else set())
    df['hobbies_sender'] = df.hobbies_sender.apply(lambda x: set(x) if isinstance(x,list) else set())
    df['hobbies_overlap'] = df.apply(lambda x: len(x['hobbies_receiver'].intersection(x['hobbies_sender'])), axis=1)

    
    df['amenities_receiver'] = df.amenities_receiver.apply(lambda x: set(x) if isinstance(x,list) else set())
    df['amenities_sender'] = df.amenities_sender.apply(lambda x: set(x) if isinstance(x,list) else set())
    df['amenities_overlap'] = df.apply(lambda x: len(x['amenities_receiver'].intersection(x['amenities_sender'])), axis=1)

    df['neighborhoods_receiver'] = df.neighborhoods_receiver.apply(lambda x: set(x) if isinstance(x,list) else set())
    df['neighborhoods_sender'] = df.neighborhoods_sender.apply(lambda x: set(x) if isinstance(x,list) else set())
    df['neighborhoods_overlap'] = df.apply(lambda x: len(x['neighborhoods_receiver'].intersection(x['neighborhoods_sender'])), axis=1)

    df['rent_overlap'] = df.apply(get_rent_range, axis=1)
    
    df['urgency_receiver'] = df.available_receiver-df.timestamp
    df['urgency_sender'] = df.available_sender-df.timestamp

    df['distance'] = df.apply(my_distance, axis=1)
    
    TF = {True: 1, False: 0, 'male':1, 'female':0, 'shared':0,'private':1,'nan':-1}
    
    col_to_TF = ['response', 
                 'has_about_sender', 'has_about_receiver', 'gender_sender','gender_receiver',
                 'facebookId_sender', 'linkedinId_sender', 'picture_sender', 'has_room_sender',
                 'type_sender','type_receiver',
                 'facebookId_receiver', 'linkedinId_receiver', 'picture_receiver', 'has_room_receiver',
                 'same_work','same_city','same_state','same_country','same_college',
                 'same_gender','same_relate','same_clean','same_night','same_student','same_smoking','same_term','same_type']
    for col in col_to_TF:
        df[col] = df[col].map(TF)
    
    print("columns with null values: {}".format(len(df.columns[df.isnull().any()])))
    return df
  
master_df = feature_time(master_df)
master_df.head()

columns with null values: 69


,convo_length,len_receiver,len_sender,mid_receiver,mid_sender,response,timestamp,uid_receiver,uid_sender,I_count_sender,...,same_country,same_college,roommate_similarity,hobbies_overlap,amenities_overlap,neighborhoods_overlap,rent_overlap,urgency_receiver,urgency_sender,distance
conversation_id,,,,,,,,,,,,,,,,,,,,,
1Jqbth3nqH0EWNOmyQmW,1,NaN,17.0,None,-Kt7F-lKnoanl54lMnUO,0,2017-09-03 14:58:30.759,0EWNOmyQmW,1Jqbth3nqH,3.0,...,1,0,1,3,0,0,175.0,NaT,11 days 16:45:31.241000,0.552752
caEwxVJXPTGyR6cCT5K8,1,NaN,9.0,None,-KhP4gs1cj3-MfN7aZkb,0,2017-04-10 22:58:15.738,caEwxVJXPT,GyR6cCT5K8,0.0,...,0,0,1,3,0,0,175.0,NaT,NaT,0.068024
caZHBg3cEbGhtafu1Ef5,1,NaN,36.0,None,-KnNpug-1BJr75KDw7lH,0,2017-06-24 06:43:51.751,caZHBg3cEb,Ghtafu1Ef5,0.0,...,0,0,0,1,3,0,350.0,NaT,NaT,NaN
ccY3lAJp8URACdaEvb0s,1,NaN,24.0,None,-KuB6nmPi4W96oF6J_pc,0,2017-09-16 19:16:51.128,ccY3lAJp8U,RACdaEvb0s,8.0,...,0,0,0,7,1,4,100.0,75 days 21:16:39.872000,25 days 05:50:03.872000,0.092504
ccYEEH0Ug7FH2hR293u1,2,26.0,16.0,-KindiRhJWOmeQ0Nt43Q,-KimFk44pXMwVVvWqAVl,1,2017-04-28 01:53:07.603,FH2hR293u1,ccYEEH0Ug7,0.0,...,0,1,1,3,2,0,50.0,NaT,NaT,0.113974


# GET CONVERSATION LENGTH DATA

In [ ]:
def drop_columns(df):
    col_to_drop = ['conv_id','convo_length','uid_sender','uid_receiver','mid_sender','mid_receiver',
                   'updated_sender','activeAt_sender',
                   'created_sender','available_sender','about_sender','birthday_sender', 'location_sender', 'work_sender',
                   'hometownCity_sender','hometownState_sender','hometownCountry_sender','college_sender','neighborhoods_sender',
                   'amenities_sender', 'hobbies_sender',
                   'updated_receiver','activeAt_receiver',
                   'created_receiver','available_receiver','about_receiver','birthday_receiver', 'location_receiver', 'work_receiver',
                   'hometownCity_receiver','hometownState_receiver','hometownCountry_receiver','college_receiver','neighborhoods_receiver',
                   'amenities_receiver', 'hobbies_receiver']
                   
    return df.drop(col_to_drop, axis=1) 

smaller_df = drop_columns(master_df)
df = df.fillna(df.mean(axis=0))
df.head()

# PICKLE

In [41]:
# mp.json_it(df,'data_features')
mp.json_it(master_df,'data_master')